In [1]:
!pip install stable-baselines3[extra]

try:
  !rm -rf boptestGymService
except:
  pass
!git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService

In [2]:
import requests
import sys
import torch
import gymnasium as gym
import os

In [3]:
sys.path.insert(0,'src/boptestGymService')

url = 'http://localhost:80'

In [4]:
from stable_baselines3 import TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from boptestGymService.boptestGymEnv import BoptestGymEnv
from boptestGymEnv import DiscretizedActionWrapper
from gym.wrappers import FlattenObservation
from boptestGymEnv import DiscretizedObservationWrapper

C:\Users\irmak\Documents\GitHub\RL_ex\src/boptestGymService\examples\test_and_plot.py:122: SyntaxWarning: invalid escape sequence '\c'
  axs[0].set_ylabel('Operative\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\src/boptestGymService\examples\test_and_plot.py:131: SyntaxWarning: invalid escape sequence '\c'
  axs[3].set_ylabel('Ambient\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\src/boptestGymService\examples\test_and_plot.py:135: SyntaxWarning: invalid escape sequence '\d'
  axt.plot(x_time, df['weaSta_reaWeaHDirNor_y'], color='gold', linestyle='-', linewidth=1, label='$\dot{Q}_rad$')
C:\Users\irmak\Documents\GitHub\RL_ex\src/boptestGymService\examples\test_and_plot.py:141: SyntaxWarning: invalid escape sequence '\d'
  axs[3].plot([],[], color='gold',        linestyle='-', linewidth=1, label='$\dot{Q}_{rad}$')


In [5]:
# Redefine reward function
class BoptestGymEnvCustomReward(BoptestGymEnv):
    def get_reward(self):
        '''Custom reward function. We use the BOPTEST `GET /kpis` API call to compute the
        total cummulative discomfort from the beginning of the episode. Note
        that this is the true value that BOPTEST uses when evaluating
        controllers.

        'tdis_tot': temp discomfort
         'idis_tot': 0,
         'ener_tot': total energy
         'cost_tot': total cost
         'emis_tot': total emission
         'pele_tot': defines the HVAC peak electrical demand.
         'pgas_tot': defines the HVAC peak gas demand.
         'pdih_tot': defines the HVAC peak district heating demand.
         'time_rat': defines the average ratio between the controller computation time and the test simulation control step. The controller computation time is measured as the time between two emulator advances
        '''
        # Compute BOPTEST core kpis
        kpis = requests.get('{0}/kpi/{1}'.format(self.url, self.testid)).json()['payload']
        # Calculate objective integrand function as the total discomfort
        objective_integrand = kpis['tdis_tot']
        # Give reward if there is not immediate increment in discomfort
        if objective_integrand == self.objective_integrand:
          reward=1
        else:
          reward=0
        # Record current objective integrand for next evaluation
        self.objective_integrand = objective_integrand
        return reward

In [6]:
test_case="multizone_office_simple_air"

In [7]:
BoptestGymEnv?

Init signature:
BoptestGymEnv(
    url='http://127.0.0.1',
    testcase='bestest_hydronic_heat_pump',
    actions=['oveHeaPumY_u'],
    observations={'reaTZon_y': (280.0, 310.0)},
    reward=['reward'],
    max_episode_length=10800,
    random_start_time=False,
    excluding_periods=None,
    regressive_period=None,
    predictive_period=None,
    start_time=0,
    warmup_period=0,
    scenario={'electricity_price': 'constant'},
    step_period=900,
    render_episodes=False,
    log_dir='C:\\Users\\irmak\\Documents\\GitHub\\RL_ex',
)
Docstring:     
BOPTEST Environment that follows gym interface.
This environment allows the interaction of RL agents with building
emulator models from BOPTEST. 
 
Init docstring:
Parameters
----------
url: string
    Rest API url for communication with the BOPTEST interface
testcase: string
    The string identifier of the testcase
actions: list
    List of strings indicating the action space. The bounds of 
    each variable from the action space the ar

In [8]:
lower_setp = 21 + 273.15
upper_setp = 24 + 273.15

env = BoptestGymEnvCustomReward(url                   = url,
                    testcase              = test_case,
                    actions               = ['hvac_oveZonSupCor_TZonCooSet_u', 
                                             'hvac_oveZonSupNor_TZonCooSet_u',
                                             'hvac_oveZonSupSou_TZonCooSet_u'],
                    observations          = {"hvac_reaZonCor_TZon_y":(lower_setp, upper_setp),
                                             "hvac_reaZonNor_TZon_y":(lower_setp, upper_setp),
                                             "hvac_reaZonSou_TZon_y":(lower_setp, upper_setp)},
                    random_start_time     = False,
                    start_time            = 154*24*3600,
                    max_episode_length    = 24*3600,
                    warmup_period         = 24*3600,
                    step_period           = 900,
                    render_episodes= True)

C:\Users\irmak\anaconda3\Lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
C:\Users\irmak\anaconda3\Lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


<Figure size 640x480 with 0 Axes>

In [9]:
log_path =os.path.join("Training", "Boptest", "Logs")
TD3_path = os.path.join("Training" , "Saved Models", "TD3_Boptest")

In [10]:
#env?

In [11]:
model = TD3('MlpPolicy', env, verbose = 1, learning_rate=0.001, gamma= 0.99, batch_size= 100, policy_delay=2)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
model.learn(total_timesteps=10)

In [13]:
model.save(TD3_path)

In [14]:
evaluate_policy(model, env, n_eval_episodes=3)

C:\Users\irmak\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\irmak\Documents\GitHub\RL_ex\src/boptestGymService\examples\test_and_plot.py:122: SyntaxWarning: invalid escape sequence '\c'
  axs[0].set_ylabel('Operative\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\src/boptestGymService\examples\test_and_plot.py:131: SyntaxWarning: invalid escape sequence '\c'
  axs[3].set_ylabel('Ambient\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\src/boptestGymService\examples\test_and_plot.py:135: SyntaxWarning: invalid escape sequence '\d'
  axt.plot(x_time, df['weaSta_reaWeaHDirNor_y'], color='gold', linestyle='-', linewidth=1, label='$\dot{Q}_rad$')
C:\Users\irm

KeyError: 'time'

<Figure size 640x480 with 0 Axes>

In [15]:
env.stop()